# Skin Color Estimation

In [ ]:
from pathlib import Path
from pprint import pprint

import cv2 as cv
import numpy as np

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

from saac.utils import cv_imshow, color_show, quadrant_bboxes, crop_bbox
from saac.models import SkinColorMeanExtractor, SkinColorModeExtractor
from saac.process import ImageEqualizer, MidJourneyProcessor

## Sample Midjourney image

In [ ]:
images_root = Path('../../data/images/raw')

img_path = list(images_root.glob('*.png'))[23]
print(img_path)

img_2x2 = cv.imread(str(img_path))
cv_imshow(img_2x2)

### Crop quadrant and get DeepFace predictions

In [ ]:
image = crop_bbox(img_2x2, quadrant_bboxes(img_2x2.shape[:2])[2])
cv_imshow(image)

In [ ]:
processor = MidJourneyProcessor()

preds = processor.image_predictions(image)
pprint(preds)
bbox = preds['bbox']
face = crop_bbox(image, bbox)
cv_imshow(face)

## Apply skin color extractors

In [ ]:
lower_quantile = 0.5
upper_quantile = 0.9
rgb_threshold = True
hist_bins = 25

mean_extractor = SkinColorMeanExtractor(
    lower_quantile=lower_quantile,
    upper_quantile=upper_quantile,
)

mode_extractor = SkinColorModeExtractor(
    lower_quantile=lower_quantile,
    upper_quantile=upper_quantile,
    hist_bins=hist_bins
)

mean_color, mean_mask = mean_extractor.extract(face)
mode_color, mode_mask = mode_extractor.extract(face)

fig, ax = plt.subplots(2, 2, figsize=(8, 8))

cv_imshow(face*mean_mask[:, :, np.newaxis], ax=ax[0,0])
color_show(mean_color, ax=ax[1, 0])
ax[0,0].set_title('Mean')

cv_imshow(face*mode_mask[:, :, np.newaxis], ax=ax[0,1])
color_show(mode_color, ax=ax[1, 1])
ax[0,1].set_title('Mode')

fig.suptitle('Skin Color Extractors');

## Apply image equalizer before extraction

In [ ]:
clip_limit = 2.0
grid_size = 8

equalizer = ImageEqualizer(
    clipLimit=clip_limit,
    grid_size=grid_size
)

image_cl = equalizer.equalize(image)

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                       
cv_imshow(image, ax=ax[0])
ax[0].set_title('Unequalized')

cv_imshow(image_cl, ax=ax[1])
ax[1].set_title('Equalized')

In [ ]:
face_cl = crop_bbox(image_cl, bbox)

mean_color, mean_mask = mean_extractor.extract(face_cl)
mode_color, mode_mask = mode_extractor.extract(face_cl)

fig, ax = plt.subplots(2, 2, figsize=(8, 8))

cv_imshow(face*mean_mask[:, :, np.newaxis], ax=ax[0,0])
color_show(mean_color, ax=ax[1, 0])
ax[0,0].set_title('Mean')

cv_imshow(face*mode_mask[:, :, np.newaxis], ax=ax[0,1])
color_show(mode_color, ax=ax[1, 1])
ax[0,1].set_title('Mode')

fig.suptitle('Skin Color Extractors (Equalized)');